# Build model V0.0

# Net Data
    
| batch=1 | subdivisions=1 | Training |
| :---: | :---: | :---: |
| decay=0.0005 | angle=0 | saturation = 1.5 |
| batch=64 | subdivisions=16 | width=608 |
| height=608 | channels=3 | momentum=0.9 |
|exposure = 1.5 | hue=.1 | learning_rate=0.001|
|burn_in=1000| max_batches = 500200| policy=steps|  
|steps=400000,450000|scales=.1,.1||
 

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, Conv2D, LeakyReLU, ZeroPadding2D , Add , UpSampling2D , concatenate
from keras.layers import concatenate
from keras.activations import linear
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad
from keras.utils import plot_model
from keras.engine.topology import Layer
from Yolo_layer import YoloLayer
from Yolo_weight_catch import Yolo_weight

Using TensorFlow backend.


# Global Variables

In [2]:
labels = ['person'       , 'bicycle'      , 'car'          , 'motorcycle',
          'airplane'     , 'bus'          , 'train'        , 'truck', 
          'boat'         , 'traffic light', 'fire hydrant' ,'stop sign',
          'parking meter', 'bench'        , 'bird'         , 'cat', 
          'dog'          , 'horse'        , 'sheep'        ,'cow' , 
          'elephant'     , 'bear'         , 'zebra'        ,'giraffe',
          'backpack'     , 'umbrella'     , 'handbag'      ,'tie',
          'suitcase'     , 'frisbee'      , 'skis'         ,'snowboard',
          'sports ball'  , 'kite'         , 'baseball bat' , 'baseball glove',
          'skateboard'   , 'surfboard'    , 'tennis racket','bottle',
          'wine glass'   , 'cup'          , 'fork'         ,'knife',
          'spoon'        , 'bowl'         , 'banana'       ,'apple',
          'sandwich'     , 'orange'       , 'broccoli'     ,'carrot',
          'hot dog'      , 'pizza'        , 'donut'        ,'cake',
          'chair'        , 'couch'        , 'potted plant' ,'bed',
          'dining table' , 'toilet'       , 'tv'           ,'laptop', 
          'mouse'        , 'remote'       , 'keyboard'     ,'cell phone',
          'microwave'    , 'oven'         , 'toaster'      ,'sink', 
          'refrigerator' , 'book'         , 'clock'        , 'vase',
          'scissors'     , 'teddy bear'   , 'hair drier'   , 'toothbrush']

IMAGE_H = None
IMAGE_W = None
grid_scales = [7,17,37]
nb_class         = len(labels)

anchors          = [10,13,16,30,33,23,30,61,62,45,59,119,116,90,156,198,373,326]

noobj_scale      = 1.0
obj_scale        = 5.0
COORD_SCALE      = 1.0
class_scale      = 1.0

batch_size       = 16
warmup_batches   = 0
TRUE_BOX_BUFFER  = 50

Epsilon          = 0.001

ALPHA = 0.1
max_grid = [480,480]
max_box_per_image = 20

# num=9
jitter=.3

truth_thresh = 1
random=1

xywh_scale = 1
ignore_thresh = .7

# Inputs

In [3]:
# net_h, net_w, 3
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3)) 
true_boxes  = Input(shape=(1, 1, 1, max_box_per_image, 4))
# grid_h, grid_w, nb_anchor, pc+bx+by+bh+bw+nb_class
true_yolo_1 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 
true_yolo_2 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 
true_yolo_3 = Input(shape=(None, None, len(anchors)//6, 5+nb_class)) 

# Prediction Layer

In [4]:
# Conv
# Layer 1

lay1_1 = Conv2D(32, (3,3), strides=(1, 1),use_bias=False, padding='same')(input_image)
lay1_2 = BatchNormalization(epsilon=Epsilon)(lay1_1)
lay1_3 = LeakyReLU(alpha=ALPHA)(lay1_2)

# Downsample

# Conv
# Layer 2

lay2_1 = ZeroPadding2D(((1,0),(1,0)))(lay1_3)
lay2_2 = Conv2D(64, (3,3), strides=(2, 2),use_bias=False, padding='valid')(lay2_1)
lay2_3 = BatchNormalization(epsilon=Epsilon)(lay2_2)
lay2_4 = LeakyReLU(alpha=ALPHA)(lay2_3)

# Conv
# Layer 3

lay3_1 = Conv2D(32, (1,1), strides=(1, 1),use_bias=False, padding='same')(lay2_4)
lay3_2 = BatchNormalization(epsilon=Epsilon)(lay3_1)
lay3_3 = LeakyReLU(alpha=ALPHA)(lay3_2)

# Conv
# Layer 4

lay4_1 = Conv2D(64, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay3_3)
lay4_2 = BatchNormalization(epsilon=Epsilon)(lay4_1)
lay4_3 = LeakyReLU(alpha=ALPHA)(lay4_2)

# Shortcut
# Layer 5

lay5_1 = Add()([lay2_4, lay4_3])
lay5_2 = linear(lay5_1)

# Downsample

# Conv
# Layer 6

lay6_1 = ZeroPadding2D(((1,0),(1,0)))(lay5_2)
lay6_2 = Conv2D(128, (3,3), strides=(2, 2),use_bias=False, padding='valid')(lay6_1)
lay6_3 = BatchNormalization(epsilon=Epsilon)(lay6_2)
lay6_4 = LeakyReLU(alpha=ALPHA)(lay6_3)

# Conv
# Layer 7

lay7_1 = Conv2D(64, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay6_4)
lay7_2 = BatchNormalization(epsilon=Epsilon)(lay7_1)
lay7_3 = LeakyReLU(alpha=ALPHA)(lay7_2)

# Conv
# Layer 8

lay8_1 = Conv2D(128, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay7_3)
lay8_2 = BatchNormalization(epsilon=Epsilon)(lay8_1)
lay8_3 = LeakyReLU(alpha=ALPHA)(lay8_2)

# Shortcut 
# Layer 9

lay9_1 = Add()([lay6_4,lay8_3])
lay9_2 = linear(lay9_1)

# Conv
# Layer 10

lay10_1 = Conv2D(64, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay9_2)
lay10_2 = BatchNormalization(epsilon=Epsilon)(lay10_1)
lay10_3 = LeakyReLU(alpha=ALPHA)(lay10_2)

# Conv
# Layer 11

lay11_1 = Conv2D(128, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay10_3)
lay11_2 = BatchNormalization(epsilon=Epsilon)(lay11_1)
lay11_3 = LeakyReLU(alpha=ALPHA)(lay11_2)

# Shortcut 
# layer 12

lay12_1 = Add()([lay9_2,lay11_3])
lay12_2 = linear(lay12_1)

# Downsample

# Conv 
# Layer 13

lay13_1 = ZeroPadding2D(((1,0),(1,0)))(lay12_2)
lay13_2 = Conv2D(256, (3,3), strides=(2, 2), use_bias=False, padding='valid')(lay13_1)
lay13_3 = BatchNormalization(epsilon=Epsilon)(lay13_2)
lay13_4 = LeakyReLU(alpha=ALPHA)(lay13_3)

# Conv
# Layer 14

lay14_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay13_4)
lay14_2 = BatchNormalization(epsilon=Epsilon)(lay14_1)
lay14_3 = LeakyReLU(alpha=ALPHA)(lay14_2)

# Conv
# Layer 15

lay15_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay14_3)
lay15_2 = BatchNormalization(epsilon=Epsilon)(lay15_1)
lay15_3 = LeakyReLU(alpha=ALPHA)(lay15_2)


# Shortcut
# Layer 16

lay16_1 = Add()([lay13_4,lay15_3])
lay16_2 = linear(lay16_1)


# Conv
# Layer 17

lay17_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay16_2)
lay17_2 = BatchNormalization(epsilon=Epsilon)(lay17_1)
lay17_3 = LeakyReLU(alpha=ALPHA)(lay17_2)

# Conv
# Layer 18

lay18_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay17_3)
lay18_2 = BatchNormalization(epsilon=Epsilon)(lay18_1)
lay18_3 = LeakyReLU(alpha=ALPHA)(lay18_2)

# Shortcut 
# Layer 19

lay19_1 = Add()([lay16_2,lay18_3])
lay19_2 = linear(lay19_1)

# Conv
# Layer 20

lay20_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay19_2)
lay20_2 = BatchNormalization(epsilon=Epsilon)(lay20_1)
lay20_3 = LeakyReLU(alpha=ALPHA)(lay20_2)

# Conv
# Layer 21

lay21_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay20_3)
lay21_2 = BatchNormalization(epsilon=Epsilon)(lay21_1)
lay21_3 = LeakyReLU(alpha=ALPHA)(lay21_2)

# Shortcut 
# Layer 22

lay22_1 = Add()([lay19_2,lay21_3])
lay22_2 = linear(lay22_1)

# Conv
# Layer 23

lay23_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay22_2)
lay23_2 = BatchNormalization(epsilon=Epsilon)(lay23_1)
lay23_3 = LeakyReLU(alpha=ALPHA)(lay23_2)

# Conv 
# Layer 24 

lay24_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay23_3)
lay24_2 = BatchNormalization(epsilon=Epsilon)(lay24_1)
lay24_3 = LeakyReLU(alpha=ALPHA)(lay24_2)


# Shortcut 
# Layer 25

lay25_1 = Add()([lay22_2,lay24_3])
lay25_2 = linear(lay25_1)


# Conv
# Layer 26

lay26_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay25_2)
lay26_2 = BatchNormalization(epsilon=Epsilon)(lay26_1)
lay26_3 = LeakyReLU(alpha=ALPHA)(lay26_2)


# Conv
# Layer 27

lay27_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay26_3)
lay27_2 = BatchNormalization(epsilon=Epsilon)(lay27_1)
lay27_3 = LeakyReLU(alpha=ALPHA)(lay27_2)

# Shortcut 
# Layer 28

lay28_1 = Add()([lay25_2,lay27_3])
lay28_2 = linear(lay28_1)

# Conv
# Layer 29

lay29_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay28_2)
lay29_2 = BatchNormalization(epsilon=Epsilon)(lay29_1)
lay29_3 = LeakyReLU(alpha=ALPHA,)(lay29_2)

# Conv
# Layer 30

lay30_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay29_3)
lay30_2 = BatchNormalization(epsilon=Epsilon)(lay30_1)
lay30_3 = LeakyReLU(alpha=ALPHA)(lay30_2)

# Shortcut 
# Layer 31

lay31_1 = Add()([lay28_2,lay30_3])
lay31_2 = linear(lay31_1)


# Conv
# Layer 32

lay32_1 = Conv2D(128, (1,1), strides=(1, 1),use_bias=False, padding='same')(lay31_2)
lay32_2 = BatchNormalization(epsilon=Epsilon)(lay32_1)
lay32_3 = LeakyReLU(alpha=ALPHA)(lay32_2)


# Conv
# Layer 33

lay33_1 = Conv2D(256, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay32_3)
lay33_2 = BatchNormalization(epsilon=Epsilon)(lay33_1)
lay33_3 = LeakyReLU(alpha=ALPHA)(lay33_2)

# Shortcut
# Layer 34

lay34_1 = Add()([lay31_2,lay33_3])
lay34_2 = linear(lay34_1)

# Conv
# Layer 35

lay35_1 = Conv2D(128, (1,1), strides=(1, 1),use_bias=False, padding='same')(lay34_2)
lay35_2 = BatchNormalization(epsilon=Epsilon)(lay35_1)
lay35_3 = LeakyReLU(alpha=ALPHA)(lay35_2)

# Conv
# Layer 36

lay36_1 = Conv2D(256, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay35_3)
lay36_2 = BatchNormalization(epsilon=Epsilon)(lay36_1)
lay36_3 = LeakyReLU(alpha=ALPHA)(lay36_2)

# Shortcut 
# Layer 37

lay37_1 = Add()([lay34_2,lay36_3])
lay37_2 = linear(lay37_1)

# Downsample

# Conv
# Layer 38

lay38_1 = ZeroPadding2D(((1,0),(1,0)))(lay37_2)
lay38_2 = Conv2D(512, (3,3), strides=(2, 2),use_bias=False, padding='valid')(lay38_1)
lay38_3 = BatchNormalization(epsilon=Epsilon)(lay38_2)
lay38_4 = LeakyReLU(alpha=ALPHA)(lay38_3)

# Conv
# Layer 39

lay39_1 = Conv2D(256, (1,1), strides=(1, 1),use_bias=False, padding='same')(lay38_4)
lay39_2 = BatchNormalization(epsilon=Epsilon)(lay39_1)
lay39_3 = LeakyReLU(alpha=ALPHA)(lay39_2)

# Conv
# Layer 40

lay40_1 = Conv2D(512, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay39_3)
lay40_2 = BatchNormalization(epsilon=Epsilon)(lay40_1)
lay40_3 = LeakyReLU(alpha=ALPHA)(lay40_2)


# Shortcut
# Layer 41

lay41_1 = Add()([lay38_4,lay40_3])
lay41_2 = linear(lay41_1)

# Conv
# Layer 42

lay42_1 = Conv2D(256, (1,1), strides=(1, 1),use_bias=False, padding='same')(lay41_2)
lay42_2 = BatchNormalization(epsilon=Epsilon)(lay42_1)
lay42_3 = LeakyReLU(alpha=ALPHA)(lay42_2)


# Conv
# Layer 43

lay43_1 = Conv2D(512, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay42_3)
lay43_2 = BatchNormalization(epsilon=Epsilon)(lay43_1)
lay43_3 = LeakyReLU(alpha=ALPHA)(lay43_2)

# Shortcut 
# Layer 44

lay44_1 = Add()([lay41_2,lay43_3])
lay44_2 = linear(lay44_1)


# Conv
# Layer 45

lay45_1 = Conv2D(256, (1,1), strides=(1, 1),use_bias=False, padding='same')(lay44_2)
lay45_2 = BatchNormalization(epsilon=Epsilon)(lay45_1)
lay45_3 = LeakyReLU(alpha=ALPHA)(lay45_2)

# Conv
# Layer 46

lay46_1 = Conv2D(512, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay45_3)
lay46_2 = BatchNormalization(epsilon=Epsilon)(lay46_1)
lay46_3 = LeakyReLU(alpha=ALPHA)(lay46_2)

# Shortcut 
# Layer 47

lay47_1 = Add()([lay44_2,lay46_3])
lay47_2 = linear(lay47_1)


# Conv
# Layer 48

lay48_1 = Conv2D(256, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay47_2)
lay48_2 = BatchNormalization(epsilon=Epsilon)(lay48_1)
lay48_3 = LeakyReLU(alpha=ALPHA)(lay48_2)

# Conv
# Layer 49

lay49_1 = Conv2D(512, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay48_3)
lay49_2 = BatchNormalization(epsilon=Epsilon)(lay49_1)
lay49_3 = LeakyReLU(alpha=ALPHA)(lay49_2)

# Shortcut
# Layer 50

lay50_1 = Add()([lay47_2,lay49_3])
lay50_2 = linear(lay50_1)

# Conv
# Layer 51

lay51_1 = Conv2D(256, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay50_2)
lay51_2 = BatchNormalization(epsilon=Epsilon)(lay51_1)
lay51_3 = LeakyReLU(alpha=ALPHA)(lay51_2)

# Conv
# Layer 52

lay52_1 = Conv2D(512, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay51_3)
lay52_2 = BatchNormalization(epsilon=Epsilon)(lay52_1)
lay52_3 = LeakyReLU(alpha=ALPHA)(lay52_2)

# Shortcut 
# Layer 53

lay53_1 = Add()([lay50_2,lay52_3])
lay53_2 = linear(lay53_1)


# Conv
# Layer 54

lay54_1 = Conv2D(256,(1,1) , strides=(1, 1), use_bias=False, padding='same')(lay53_2)
lay54_2 = BatchNormalization(epsilon=Epsilon)(lay54_1)
lay54_3 = LeakyReLU(alpha=ALPHA)(lay54_2)

# Conv
# Layer 55

lay55_1 = Conv2D(512, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay54_3)
lay55_2 = BatchNormalization(epsilon=Epsilon)(lay55_1)
lay55_3 = LeakyReLU(alpha=ALPHA)(lay55_2)

# Shortcut 
# Layer 56

lay56_1 = Add()([lay53_2,lay55_3])
lay56_2 = linear(lay56_1)


# Conv
# Layer 57

lay57_1 = Conv2D(256, 1, strides=(1, 1), use_bias=False, padding='same')(lay56_2)
lay57_2 = BatchNormalization(epsilon=Epsilon)(lay57_1)
lay57_3 = LeakyReLU(alpha=ALPHA)(lay57_2)

# Conv
# Layer 58

lay58_1 = Conv2D(512, 3, strides=(1, 1), use_bias=False, padding='same')(lay57_3)
lay58_2 = BatchNormalization(epsilon=Epsilon)(lay58_1)
lay58_3 = LeakyReLU(alpha=ALPHA)(lay58_2)

# Shortcut 
# Layer 59

lay59_1 = Add()([lay56_2,lay58_3])
lay59_2 = linear(lay59_1)


# Conv
# Layer 60

lay60_1 = Conv2D(256, 1, strides=(1, 1), use_bias=False, padding='same')(lay59_2)
lay60_2 = BatchNormalization(epsilon=Epsilon)(lay60_1)
lay60_3 = LeakyReLU(alpha=ALPHA)(lay60_2)

# Conv
# Layer 61

lay61_1 = Conv2D(512, 3, strides=(1, 1), use_bias=False, padding='same')(lay60_3)
lay61_2 = BatchNormalization(epsilon=Epsilon)(lay61_1)
lay61_3 = LeakyReLU(alpha=ALPHA)(lay61_2)

# Shortcut 
# Layer 62

lay62_1 = Add()([lay59_2,lay61_3])
lay62_2 = linear(lay62_1)

# Downsample

# Conv
# Layer 63

lay63_1 = ZeroPadding2D(((1,0),(1,0)))(lay62_2)
lay63_2 = Conv2D(1024, 3, strides=(2, 2), use_bias=False, padding='valid')(lay63_1)
lay63_3 = BatchNormalization(epsilon=Epsilon)(lay63_2)
lay63_4 = LeakyReLU(alpha=ALPHA)(lay63_3)

# Conv
# Layer 64

lay64_1 = Conv2D(512, 1, strides=(1, 1), use_bias=False, padding='same')(lay63_4)
lay64_2 = BatchNormalization(epsilon=Epsilon)(lay64_1)
lay64_3 = LeakyReLU(alpha=ALPHA)(lay64_2)

# Conv
# Layer 65

lay65_1 = Conv2D(1024, 3, strides=(1, 1), use_bias=False, padding='same')(lay64_3)
lay65_2 = BatchNormalization(epsilon=Epsilon)(lay65_1)
lay65_3 = LeakyReLU(alpha=ALPHA)(lay65_2)

# Shortcut
# Layer 66

lay66_1 = Add()([lay65_3,lay63_4])
lay66_2 = linear(lay66_1)

# Conv
# Layer 67

lay67_1 = Conv2D(512, 1, strides=(1, 1), use_bias=False, padding='same')(lay66_2)
lay67_2 = BatchNormalization(epsilon=Epsilon)(lay67_1)
lay67_3 = LeakyReLU(alpha=ALPHA)(lay67_2)

# Conv
# Layer 68

lay68_1 = Conv2D(1024, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay67_3)
lay68_2 = BatchNormalization(epsilon=Epsilon)(lay68_1)
lay68_3 = LeakyReLU(alpha=ALPHA)(lay68_2)

# Shortcut 
# Layer 69

lay69_1 = Add()([lay68_3,lay66_2])
lay69_2 = linear(lay69_1)

# Conv
# Layer 70

lay70_1 = Conv2D(512, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay69_2)
lay70_2 = BatchNormalization(epsilon=Epsilon)(lay70_1)
lay70_3 = LeakyReLU(alpha=ALPHA)(lay70_2)

# Conv
# Layer 71

lay71_1 = Conv2D(1024, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay70_3)
lay71_2 = BatchNormalization(epsilon=Epsilon)(lay71_1)
lay71_3 = LeakyReLU(alpha=ALPHA)(lay71_2)

# Shortcut
# Layer 72

lay72_1 = Add()([lay71_3,lay69_2])
lay72_2 = linear(lay72_1)


# Conv
# Layer 73

lay73_1 = Conv2D(512, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay72_2)
lay73_2 = BatchNormalization(epsilon=Epsilon)(lay73_1)
lay73_3 = LeakyReLU(alpha=ALPHA)(lay73_2)


# Conv
# Layer 74

lay74_1 = Conv2D(1024, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay73_3)
lay74_2 = BatchNormalization(epsilon=Epsilon)(lay74_1)
lay74_3 = LeakyReLU(alpha=ALPHA)(lay74_2)

# Shortcut 
# Layer 75

lay75_1 = Add()([lay74_3,lay72_2])
lay75_2 = linear(lay75_1)

# ######################

# Conv
# Layer 76

lay76_1 = Conv2D(512, 1, strides=(1, 1),use_bias=False, padding='same')(lay75_2)
lay76_2 = BatchNormalization(epsilon=Epsilon)(lay76_1)
lay76_3 = LeakyReLU(alpha=ALPHA)(lay76_2)

# Conv
# Layer 77

lay77_1 = Conv2D(1024, 3, strides=(1, 1),use_bias=False, padding='same')(lay76_3)
lay77_2 = BatchNormalization(epsilon=Epsilon)(lay77_1)
lay77_3 = LeakyReLU(alpha=ALPHA)(lay77_2)

# Conv
# Layer 78

lay78_1 = Conv2D(512, 1, strides=(1, 1),use_bias=False, padding='same')(lay77_3)
lay78_2 = BatchNormalization(epsilon=Epsilon)(lay78_1)
lay78_3 = LeakyReLU(alpha=ALPHA)(lay78_2)

# Conv
# Layer 79

lay79_1 = Conv2D(1024, 3, strides=(1, 1),use_bias=False, padding='same')(lay78_3)
lay79_2 = BatchNormalization(epsilon=Epsilon)(lay79_1)
lay79_3 = LeakyReLU(alpha=ALPHA)(lay79_2)

# Conv
# Layer 80

lay80_1 = Conv2D(512, 1, strides=(1, 1),use_bias=False, padding='same')(lay79_3)
lay80_2 = BatchNormalization(epsilon=Epsilon)(lay80_1)
lay80_3 = LeakyReLU(alpha=ALPHA)(lay80_2)

# Conv
# Layer 81

lay81_1 = Conv2D(1024, 3, strides=(1, 1),use_bias=False, padding='same')(lay80_3)
lay81_2 = BatchNormalization(epsilon=Epsilon)(lay81_1)
lay81_3 = LeakyReLU(alpha=ALPHA)(lay81_2)

# Conv
# Layer 82

lay82 = Conv2D(255, 1, strides=(1, 1),use_bias=True, padding='same')(lay81_3)
#lay82_2 = BatchNormalization(epsilon=Epsilon)(lay82_1)
#lay82_3 = LeakyReLU(alpha=ALPHA)(lay82_2)

# YOLO Detection Layer
# Layer 83

# Prediction Layer in First Scale
# mask = 6,7,8

#pred_yolo_1 = Conv2D((3*(5+nb_class)), 1, strides=(1, 1), padding='same', use_bias='True')(lay82_3)


# Loss Layer in First Scale
loss_yolo_1 = YoloLayer(anchors[12:], 
                        [1*num for num in max_grid], 
                        batch_size, 
                        warmup_batches, 
                        ignore_thresh, 
                        grid_scales[0],
                        obj_scale,
                        noobj_scale,
                        xywh_scale,
                        class_scale)([input_image, lay82, 
                                      true_yolo_1, true_boxes])


# Route (Skip)
# Layer 84

lay84=lay80_3

# Conv
# Layer 85

lay85_1 = Conv2D(256, 1, strides=(1, 1),use_bias=False, padding='same')(lay84)
lay85_2 = BatchNormalization(epsilon=Epsilon)(lay85_1)
lay85_3 = LeakyReLU(alpha=ALPHA)(lay85_2)

# [upsample] 86
# stride=2

lay86_1 = UpSampling2D(size=(2,2))(lay85_3)

# [route] 87 Concatenante
# layers = -1, 61

lay87_1 = concatenate([lay86_1,lay62_2],axis=-1)

# Conv
# Layer 88

lay88_1 = Conv2D(256, 1, strides=(1, 1),use_bias=False, padding='same')(lay87_1)
lay88_2 = BatchNormalization(epsilon=Epsilon)(lay88_1)
lay88_3 = LeakyReLU(alpha=ALPHA)(lay88_2)

# Conv
# Layer 89

lay89_1 = Conv2D(512, 3, strides=(1, 1),use_bias=False, padding='same')(lay88_3)
lay89_2 = BatchNormalization(epsilon=Epsilon)(lay89_1)
lay89_3 = LeakyReLU(alpha=ALPHA)(lay89_2)

# Conv
# Layer 90

lay90_1 = Conv2D(256, 1, strides=(1, 1),use_bias=False, padding='same')(lay89_3)
lay90_2 = BatchNormalization(epsilon=Epsilon)(lay90_1)
lay90_3 = LeakyReLU(alpha=ALPHA)(lay90_2)

# Conv
# Layer 91

lay91_1 = Conv2D(512, 3, strides=(1, 1),use_bias=False, padding='same')(lay90_3)
lay91_2 = BatchNormalization(epsilon=Epsilon)(lay91_1)
lay91_3 = LeakyReLU(alpha=ALPHA)(lay91_2)

# Conv
# Layer 92

lay92_1 = Conv2D(256, 1, strides=(1, 1),use_bias=False, padding='same')(lay91_3)
lay92_2 = BatchNormalization(epsilon=Epsilon)(lay92_1)
lay92_3 = LeakyReLU(alpha=ALPHA)(lay92_2)

# Conv
# Layer 93

lay93_1 = Conv2D(512, 3, strides=(1, 1),use_bias=False, padding='same')(lay92_3)
lay93_2 = BatchNormalization(epsilon=Epsilon)(lay93_1)
lay93_3 = LeakyReLU(alpha=ALPHA)(lay93_2)

# Conv
# Layer 94

lay94 = Conv2D(255, 1, strides=(1, 1),use_bias=True, padding='same')(lay93_3)
#lay94_2 = BatchNormalization(epsilon=Epsilon)(lay94_1)
#lay94_3 = LeakyReLU(alpha=ALPHA)(lay94_2)

# YOLO Detection Layer
# Layer 95

# Prediction Layer in First Scale
# mask = 3,4,5


#pred_yolo_2 = Conv2D((3*(5+nb_class)), 1, strides=(1, 1), padding='same', use_bias='True')(lay94_3)

loss_yolo_2 = YoloLayer(anchors[6:12], 
                        [1*num for num in max_grid], 
                        batch_size, 
                        warmup_batches, 
                        ignore_thresh, 
                        grid_scales[1],
                        obj_scale,
                        noobj_scale,
                        xywh_scale,
                        class_scale)([input_image, lay94, 
                                      true_yolo_2, true_boxes])
# Route (Skip)
# Layer 96

lay96=lay92_3

# Conv
# Layer 97

lay97_1 = Conv2D(128, 1, strides=(1, 1),use_bias=False, padding='same')(lay96)
lay97_2 = BatchNormalization(epsilon=Epsilon)(lay97_1)
lay97_3 = LeakyReLU(alpha=ALPHA)(lay97_2)

# Upsample
# Layer 98
# stride=2

lay98 = UpSampling2D(size=(2,2))(lay97_3)

# [route] 99
# layers = -1, 36

lay99 = concatenate([lay98,lay37_2],axis=-1)

# Conv 
# Layer 100

lay100_1 = Conv2D(128, 1, strides=(1, 1),use_bias=False, padding='same')(lay99)
lay100_2 = BatchNormalization(epsilon=Epsilon)(lay100_1)
lay100_3 = LeakyReLU(alpha=ALPHA)(lay100_2)

# Conv
# Layer 101

lay101_1 = Conv2D(256, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay100_3)
lay101_2 = BatchNormalization(epsilon=Epsilon)(lay101_1)
lay101_3 = LeakyReLU(alpha=ALPHA)(lay101_2)

# Conv
# Layer 102

lay102_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay101_3)
lay102_2 = BatchNormalization(epsilon=Epsilon)(lay102_1)
lay102_3 = LeakyReLU(alpha=ALPHA)(lay102_2)

# Conv
# Layer 103

lay103_1 = Conv2D(256, (3,3), strides=(1, 1), use_bias=False, padding='same')(lay102_3)
lay103_2 = BatchNormalization(epsilon=Epsilon)(lay103_1)
lay103_3 = LeakyReLU(alpha=ALPHA)(lay103_2)

# Conv
# Layer 104

lay104_1 = Conv2D(128, (1,1), strides=(1, 1), use_bias=False, padding='same')(lay103_3)
lay104_2 = BatchNormalization(epsilon=Epsilon)(lay104_1)
lay104_3 = LeakyReLU(alpha=ALPHA)(lay104_2)

# Conv
# Layer 105

lay105_1 = Conv2D(256, (3,3), strides=(1, 1),use_bias=False, padding='same')(lay104_3)
lay105_2 = BatchNormalization(epsilon=Epsilon)(lay105_1)
lay105_3 = LeakyReLU(alpha=ALPHA)(lay105_2)

# Conv
# Layer 106

lay106 = Conv2D(255, (1,1), strides=(1, 1), use_bias=True, padding='same')(lay105_3)
#lay106_2 = BatchNormalization(epsilon=Epsilon)(lay106_1)
#lay106_3 = LeakyReLU(alpha=ALPHA)(lay106_2)


# [yolo] 
# Layer 107
# mask = 0,1,2
# pred_yolo_3 = Conv2D((3*(5+nb_class)), 1, strides=(1, 1), padding='same', use_bias='True')(lay106_3)
loss_yolo_3 = YoloLayer(anchors[:6], 
                        [1*num for num in max_grid], 
                        batch_size, 
                        warmup_batches, 
                        ignore_thresh, 
                        grid_scales[2],
                        obj_scale,
                        noobj_scale,
                        xywh_scale,
                        class_scale)([input_image, lay106, 
                                      true_yolo_3, true_boxes])

In [5]:
train_model = Model([input_image, true_boxes, true_yolo_1, true_yolo_2, true_yolo_3], [loss_yolo_1, loss_yolo_2, loss_yolo_3])
infer_model = Model(input_image, [lay82, lay94, lay106])

plot_model(train_model, to_file='train_model.png',show_shapes='true')
plot_model(infer_model, to_file='infer_model.png',show_shapes='true')

In [6]:
print(infer_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [7]:
#print(train_model.summary())

In [8]:
fp      = open("./yolov3.weights", "rb")
header  = np.fromfile(fp, dtype = np.int32, count = 5)
weights = np.fromfile(fp, dtype = np.float32)

In [9]:
print(weights.size)

62001757
